# Amazon Personalize : utilisation du texte comme métadonnées d'article non structurés

La pertinence des recommandations que vous fournissez avec Amazon Personalize dépend des données disponibles au moment où les recommandations sont générées. Amazon Personalize utilise l'historique des interactions de vos utilisateurs, les attributs de vos articles et les métadonnées de vos utilisateurs pour déterminer quels articles sont les plus pertinents pour chaque utilisateur. Les principales données requises par Amazon Personalize sont les interactions utilisateur-article. Les interactions des utilisateurs avec les articles de votre catalogue, comme un clic sur un produit, la lecture d'un article, le visionnage d'une vidéo ou l'achat d'un produit, sont un signal important de ce qu'ils ont trouvé pertinent dans le passé. L'inclusion d'attributs d'articles et d'utilisateurs, également appelés métadonnées, peut améliorer la pertinence des recommandations, en particulier pour les nouveaux articles qui sont similaires à ceux que vos utilisateurs ont trouvés pertinents. Cependant, les métadonnées structurées telles que la catégorie, le style ou le genre d'un article ne sont pas toujours facilement disponibles ou ne fournissent pas toutes les informations que vous possédez dans vos descriptions narratives. Désormais, Amazon Personalize vous permet d'ajouter des métadonnées non structurées telles que des descriptions de produits, des transcriptions de vidéos ou des textes d'articles avec vos autres attributs d'articles. Amazon Personalize héberge, gère et utilise automatiquement des modèles de traitement du langage naturel (NLP) pour traiter votre texte et l'utiliser pour améliorer les performances de vos solutions Amazon Personalize.

Ce bloc-notes montre comment le texte sous forme de descriptions de produits peut être inclus comme métadonnées d'articles non structurées pour améliorer la pertinence des recommandations.

Les données Amazon Reviews de la catégorie Amazon Prime Pantry sont utilisées pour les jeux de données des interactions et des articles.

Lorsque vous envisagez d'inclure du texte dans votre jeu de données d'articles, gardez à l'esprit les bonnes pratiques suivantes.
- Un texte validé éditorialement pour être concis, pertinent et informatif pour chaque article, où les détails les plus pertinents sont mentionnés au début du texte, est préférable à un contenu généré par l'utilisateur qui peut être moins pertinent ou cohérent.
- Une colonne de texte peu remplie diminuera l'impact positif de l'inclusion de texte dans le jeu de données des articles.
- Nettoyer tout le texte des identifications et supprimer les espaces superflus avant de l'ajouter au jeu de données des articles.
- L'anglais est actuellement la seule langue prise en charge pour les champs de texte.
- Les champs de texte ne sont actuellement pris en compte que pour les recettes User-Personalization et Personalized-Ranking.

Deux groupes de jeux de données seront créés, qui comprendront des données avec et sans description des articles, afin que nous puissions entraîner des modèles distincts et comparer leurs résultats hors ligne et en ligne.

In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import boto3
import time
from time import sleep
from lxml import html

## Charger et inspecter les jeux de données

Nous allons commencer par charger le jeu de données d'avis sur Prime Pantry. Vous devrez remplir le formulaire pour avoir accès aux fichiers de données :

http://deepyeti.ucsd.edu/jianmo/amazon/index.html

Citation :
> Justification des avis à l'aide des évaluations à distance et des aspects précis  
> Jianmo Ni, Jiacheng Li, Julian McAuley  
> Empirical Methods in Natural Language Processing (EMNLP), 2019 [pdf](http://cseweb.ucsd.edu/~jmcauley/pdfs/emnlp19a.pdf)

In [2]:
data_dir = 'raw_data'
!mkdir $data_dir

!cd $data_dir && \
    wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Prime_Pantry.json.gz && \
    wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Prime_Pantry.json.gz

mkdir: cannot create directory ‘raw_data’: File exists
--2021-07-13 22:06:52--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Prime_Pantry.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45435146 (43M) [application/octet-stream]
Saving to: ‘Prime_Pantry.json.gz’

Prime_Pantry.json.g 100%[===================>]  43.33M  12.3MB/s    in 4.8s    

2021-07-13 22:06:57 (9.01 MB/s) - ‘Prime_Pantry.json.gz’ saved [45435146/45435146]

--2021-07-13 22:06:57--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Prime_Pantry.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5281662 (5.0M) [application/octet-stream]
Saving to: ‘meta_Prime_Pantry.json.gz’

meta_Prime_Pa

### Charger et inspecter les données des avis

Nous allons commencer par charger le jeu de données d'avis sur les produits Prime Pantry et exécuter quelques commandes pour déterminer avec quoi nous devons travailler.

In [3]:
pantry_df = pd.read_json(data_dir + '/Prime_Pantry.json.gz', lines=True, compression='infer')
pantry_df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5,True,"12 14, 2014",A1NKJW0TNRVS7O,B0000DIWNZ,Tamara M.,Good clinging,Clings well,1418515200,NaN,NaN,NaN
1,4,True,"11 20, 2014",A2L6X37E8TFTCC,B0000DIWNZ,Amazon Customer,Fantastic buy and a good plastic wrap. Even t...,Saran could use more Plus to Cling better.,1416441600,NaN,NaN,NaN
2,4,True,"10 11, 2014",A2WPR4W6V48121,B0000DIWNZ,noname,ok,Four Stars,1412985600,NaN,NaN,NaN
3,3,False,"09 1, 2014",A27EE7X7L29UMU,B0000DIWNZ,ZapNZs,Saran Cling Plus is kind of like most of the C...,"The wrap is fantastic, but the dispensing, cut...",1409529600,4,NaN,NaN
4,4,True,"08 10, 2014",A1OWT4YZGB5GV9,B0000DIWNZ,Amy Rogers,This is my go to plastic wrap so there isn't m...,has been doing it's job for years,1407628800,NaN,NaN,NaN


In [4]:
pantry_df.shape

(471614, 12)

Que pouvons-nous apprendre de ce résultat ? Il y a plus de 471 000 avis et 12 colonnes de données. La colonne `asin` est notre identificateur unique de l'article, `reviewerID` est notre identificateur unique de l'utilisateur, `unixReviewTime` est l'horodatage du commentaire et `overall` indique le niveau de positivité de l'avis sur une échelle de 1 à 5. Nous utiliserons ce fichier comme base pour notre jeu de données d'interactions pour Personalize. 

### Créer et enregistrer un jeu de données d'interactions

Commençons par créer notre jeu de données d'interactions en sélectionnant les lignes que nous voulons inclure. La première étape est d'isoler uniquement les avis positifs. Pour ce faire, nous supposons que les avis ayant une note globale de 4 ou plus est positif. Toute note de 3 ou moins est soit médiocre, soit négative.

In [5]:
positive_reviews_df = pantry_df[pantry_df['overall'] > 3]
positive_reviews_df.shape

(387692, 12)

Nous obtenons 387 000 avis positifs. C'est encore beaucoup pour entraîner un modèle dans Personalize.

Ensuite, nous allons réduire le jeu de données aux seules colonnes dont nous avons besoin et ajouter une colonne `EVENT_TYPE` pour indiquer le type d'événements que nous capturons. En ajoutant une colonne `EVENT_TYPE` maintenant, vous pourrez plus facilement explorer les tests d'événements en temps réel plus tard si vous choisissez de le faire (puisque `eventType` est un champ obligatoire pour l'API [PutEvents](https://docs.aws.amazon.com/personalize/latest/dg/API_UBS_PutEvents.html)).

In [6]:
positive_reviews_df = positive_reviews_df[['reviewerID', 'asin', 'unixReviewTime', 'overall']]
positive_reviews_df['EVENT_TYPE']='reviewed'

positive_reviews_df.head()

,reviewerID,asin,unixReviewTime,overall,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
4,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
5,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed


Nous devons faire une dernière vérification, à savoir contrôler la validité d'une valeur de la colonne `unixReviewTime`. Comme Personalize crée des modèles de séquence basés sur la date et l'heure de chaque interaction, il est important que l'horodatage de chaque interaction soit représenté dans le format attendu, afin qu'il soit interprété correctement.

Choisissons une valeur pour la colonne `unixReviewTime` et analysons-la en une date lisible par l'homme afin de vérifier qu'elle est raisonnable.

In [7]:
time_stamp = positive_reviews_df.iloc[50]['unixReviewTime']
print(time_stamp)
print(datetime.utcfromtimestamp(time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1321488000
2011-11-17 00:00:00


La valeur de l'horodatage semble correcte. Obtenons quelques informations récapitulatives finales pour notre jeu de données.

In [8]:
positive_reviews_df.describe(include='all')

,reviewerID,asin,unixReviewTime,overall,EVENT_TYPE
count,387692,387692,3.876920e+05,387692.000000,387692
unique,202254,10584,NaN,NaN,1
top,A35Q0RBM3YNQNF,B00XA9DADC,NaN,NaN,reviewed
freq,176,5288,NaN,NaN,387692
mean,NaN,NaN,1.468847e+09,4.847227,NaN
std,NaN,NaN,4.314975e+07,0.359769,NaN
min,NaN,NaN,1.073693e+09,4.000000,NaN
25%,NaN,NaN,1.447200e+09,5.000000,NaN
50%,NaN,NaN,1.474718e+09,5.000000,NaN
75%,NaN,NaN,1.498435e+09,5.000000,NaN


Nous avons 387 000 avis pour 202 000 évaluateurs/utilisateurs distincts sur 10 000 produits uniques. C'est la base de notre jeu de données d'interactions.

Pour pouvoir l'utiliser comme jeu de données d'interactions, nous devons toutefois renommer les colonnes pour qu'elles correspondent à celles attendues par Personalize.

In [9]:
positive_reviews_df.rename(columns = {'reviewerID':'USER_ID', 'asin':'ITEM_ID', 
                              'unixReviewTime':'TIMESTAMP', 'overall': 'EVENT_VALUE'}, inplace = True)
positive_reviews_df

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_VALUE,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
4,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
5,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed
...,...,...,...,...,...
471609,A19GSVHXVT5NNF,B01HI8JVI8,1494892800,5,reviewed
471610,ABSCTKLX9F9IU,B01HI8JVI8,1493769600,5,reviewed
471611,A2R33RCWKDHZ3L,B01HI8JVI8,1492646400,5,reviewed
471612,A2INGHYEXZDHMC,B01HI8JVI8,1492560000,5,reviewed


Enfin, sauvegardons notre cadre de données des avis positifs au format CSV. Nous chargerons le fichier CSV sur Personalize plus tard dans ce bloc-notes.

In [10]:
interactions_filename = "interactions.csv"
positive_reviews_df.to_csv(interactions_filename, index=False, float_format='%.0f')

### Charger et inspecter les métadonnées des articles

Maintenant que nous avons établi le jeu de données des interactions, passons au jeu de données des articles. C'est là que nous trouverons la valeur du texte non structuré que nous allons inclure dans le modèle.

À l'instar du jeu de données des avis, le fichier de métadonnées des articles Prime Pantry est également représenté en JSON. En raison de la nature imbriquée de ce fichier, il sera difficile de formater nos données comme nous le souhaitons.

Commençons par charger le fichier de métadonnées dans un cadre de données et examinons les données.

In [11]:
pantry_meta_df = pd.read_json('raw_data/meta_Prime_Pantry.json.gz', lines=True, compression='infer')
pantry_meta_df

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,NaT,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,NaT,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,[],,[A perfect Lentil soup starts with Goya Lentil...,,"Goya Dry Lentils, 16 oz","[B003SI144W, B000VDRKEK]",,Goya,[],[],"[B074MFVZG7, B079PTH69L, B000VDRKEK, B074M9T81...",{'ASIN: ': 'B0000DIF38'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIF38,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,[],,[Saran Premium Wrap is an extra tough yet easy...,,"Saran Premium Plastic Wrap, 100 Sq Ft","[B01MY5FHT6, B000PYF8VM, B000SRMDFA, B07CX6LN8...",,Saran,[],[],"[B077QLSLRQ, B00JPKW1RQ, B000FE2IK6, B00XUJHJ9...",{'Domestic Shipping: ': 'This item can only be...,"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIWNI,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,,"Saran Cling Plus Plastic Wrap, 200 Sq Ft",[],,Saran,[],[],[B0014CZ0TE],{'Domestic Shipping: ': 'This item can only be...,"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIWNZ,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10808,[],,[These bars are where our journey started and ...,,"KIND Bars, Caramel Almond &amp; Sea Salt, Glut...",[],,KIND,[],"26,259 in Grocery & Gourmet Food (","[B00JQQAN60, B00JQQAWSY, B0111K7V54, B0111K8L9...","{'ASIN: ': 'B01HI76312', 'Item model number:':...","<img src=""https://images-na.ssl-images-amazon....",,NaT,$3.98,B01HI76312,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
10809,[],,[These bars are where our journey started and ...,,"KIND Bars, Maple Glazed Pecan &amp; Sea Salt, ...",[],,KIND,[],"16,822 in Grocery & Gourmet Food (","[B0111K97JC, B00JQQAN60, B0111K8L9Y, B01HI7631...",{'ASIN: ': 'B01HI76790'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$5.81,B01HI76790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
10810,[],,[These bars are where our journey started and ...,,"KIND Bars, Dark Chocolate Almond &amp; Coconut...",[],,KIND,[],"107,057 in Grocery & Gourmet Food (","[B0111K7V54, B01HI76312, B00JQQAL0S, B0111K97J...",{'ASIN: ': 'B01HI76SA8'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$4.98,B01HI76SA8,[],[]
10811,[],,[These bars are where our journey started and ...,,"KIND Bars, Honey Roasted Nuts &amp; Sea Salt, ...",[],,KIND,[],"24,648 in Grocery & Gourmet Food (","[B00JQQAN60, B0111K7V54, B01HI76312, B0111K97J...",{'ASIN: ': 'B01HI76XS0'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$5.81,B01HI76XS0,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [12]:
pantry_meta_df.describe()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
count,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,0,10813,10813,10813,10813
unique,1,1,9409,1,10782,3957,1,1960,763,4828,5940,10786,4,1,0,1482,10812,8940,8940
top,[],,[],,"Infants' Motrin Concentrated Drops, Fever Redu...",[],,L'Oreal Paris,[],[],[],{},"<img src=""https://images-na.ssl-images-amazon....",,NaN,,B00005BPJO,[],[]
freq,10813,10813,98,10813,2,6754,10813,171,9777,5937,4835,24,10621,10813,NaN,4063,2,1781,1781


Que pouvons-nous donc apprendre de ces informations ? Tout d'abord, il y a plus de 10 000 produits représentés dans le fichier de métadonnées. La plupart des colonnes ne nous seront pas d'une grande utilité pour Personalize, car elles ne sont pas pertinentes en tant que caractéristiques (URL des images, `details`, `also_viewed`, `also_buy`, etc.) ou pour la plupart sont vides/contiennent peu de données (`category`, `fit`, `tech1`, etc.). La colonne `asin` est notre identificateur unique pour chaque article (bien qu'il semble exister un doublon) et `brand` et `price` semblent pouvoir être utiles. Nous allons utiliser la colonne `description` pour le texte non structuré.

Cependant, nous devons nettoyer et reformater les champs que nous voulons utiliser dans notre jeu de données des articles. Par exemple, le champ `price` est une valeur monétaire formatée (chaîne de caractères) et non numérique et le champ `description` a été chargé comme un tableau de chaînes de caractères en raison de la façon dont les valeurs ont été représentées et analysées à partir du fichier JSON d'origine. Enfin, les valeurs `description` contiennent également des balises HTML qui doivent être supprimées.

Commençons par créer un cadre de données avec uniquement les colonnes dont nous avons besoin pour le jeu de données des articles.

In [13]:
items_df = pantry_meta_df.copy()
items_df = items_df[['asin', 'brand', 'price', 'description']]
items_df.head(10)

,asin,brand,price,description
0,B00005BPJO,Milk Duds,$5.00,[Sink your sweet tooth into MILK DUDS Candya d...
1,B00005BPJO,Milk Duds,$5.00,[Sink your sweet tooth into MILK DUDS Candya d...
2,B0000DIF38,Goya,,[A perfect Lentil soup starts with Goya Lentil...
3,B0000DIWNI,Saran,,[Saran Premium Wrap is an extra tough yet easy...
4,B0000DIWNZ,Saran,,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...
5,B0000GH6UG,Ibarra,,"[Ibarra Chocolate, 19 Oz, , ]"
6,B0000KC2BK,Knorr,$3.09,[Knorr Granulated Chicken Flavor Bouillon is a...
7,B0001E1IN8,Castillo,,[Red chili habanero sauces. They are present t...
8,B00032E8XK,Chicken of the Sea,$1.48,[Chicken of the Sea Solid White Albacore Tuna ...
9,B0005XMTHE,Smucker's,$2.29,"[Helps build muscles with bcaa's amino acids, ..."


Ensuite, nous allons supprimer les lignes en double en fonction de la valeur de la colonne `asin`. Il ne doit exister qu'un seul doublon d'après la sortie `describe()` ci-dessus.

In [14]:
items_df = items_df.drop_duplicates(subset=['asin'], keep='last')
items_df.shape

(10812, 4)

Ensuite, concentrons-nous sur le reformatage et le nettoyage des valeurs de la colonne `description`. Comme vous pouvez le voir ci-dessus, la `description` est actuellement représentée sous la forme d'un tableau de chaînes (car c'est ainsi qu'elle est représentée dans le fichier JSON). Nous devons transformer ce tableau en une chaîne unique et effacer toutes les balises HTML de chaque fragment.

Nous allons commencer par créer deux fonctions utilitaires qui seront utilisées pour nettoyer la colonne `description` (et ensuite la colonne `title` dans le jeu de données d'origine lorsque nous voudrons afficher les titres des produits recommandés).

In [15]:
# Strips and cleans a value of HTML markup and whitespace.
def clean_markup(value):
    s = str(value).strip()
    if s != '':
        s = str(html.fromstring(s).text_content())
        s = ' '.join(s.split())
                
    return s.strip()

# Cleans and reformats the description column value for a dataframe row.
def clean_and_reformat_description(row):
    s = ''
    for el in row['description']:
        el = clean_markup(el)
        if el != '':
            s += ' ' + el
                
    return s.strip()

In [16]:
items_df['description'] = items_df.apply(clean_and_reformat_description, axis=1)
items_df

,asin,brand,price,description
1,B00005BPJO,Milk Duds,$5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,,"Ibarra Chocolate, 19 Oz"
...,...,...,...,...
10808,B01HI76312,KIND,$3.98,These bars are where our journey started and i...
10809,B01HI76790,KIND,$5.81,These bars are where our journey started and i...
10810,B01HI76SA8,KIND,$4.98,These bars are where our journey started and i...
10811,B01HI76XS0,KIND,$5.81,These bars are where our journey started and i...


Ensuite, examinons la colonne `price` et convertissons son type de String en Float.

In [17]:
items_df['price'].value_counts()

          4063
$2.99      114
$3.99      113
$4.99      103
$5.99       87
          ... 
$20.42       1
$32.32       1
$1.52        1
$27.89       1
$39.10       1
Name: price, Length: 1482, dtype: int64

La cellule suivante va convertir les prix vides/non numériques en `np.nan`, et le symbole de de devise `$` sera supprimé pour tous les autres. Cela nous permettra de convertir le type en un flottant.

In [18]:
def convert_price(row):
    v = str(row['price']).strip().replace('$', '')
    if v == '' or not v.lstrip('-').replace('.', '').isdigit():
        return np.nan
    return v

items_df['price'] = items_df.apply(convert_price, axis=1)
items_df

,asin,brand,price,description
1,B00005BPJO,Milk Duds,5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,NaN,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,NaN,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,NaN,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,NaN,"Ibarra Chocolate, 19 Oz"
...,...,...,...,...
10808,B01HI76312,KIND,3.98,These bars are where our journey started and i...
10809,B01HI76790,KIND,5.81,These bars are where our journey started and i...
10810,B01HI76SA8,KIND,4.98,These bars are where our journey started and i...
10811,B01HI76XS0,KIND,5.81,These bars are where our journey started and i...


In [19]:
items_df['price'].value_counts()

2.99     114
3.99     113
4.99     103
5.99      87
2.98      76
        ... 
39.10      1
1.84       1
22.95      1
12.17      1
11.09      1
Name: price, Length: 1480, dtype: int64

In [20]:
items_df['price'] = items_df['price'].astype(float)

Ensuite, nous allons renommer les colonnes pour qu'elles correspondent aux noms et au format de nom en majuscules attendus par Personalize.

In [21]:
items_df.rename(columns = {'asin':'ITEM_ID', 'brand':'BRAND', 
                              'price':'PRICE', 'description': 'DESCRIPTION'}, inplace = True)
items_df.head(10)

,ITEM_ID,BRAND,PRICE,DESCRIPTION
1,B00005BPJO,Milk Duds,5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,NaN,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,NaN,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,NaN,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,NaN,"Ibarra Chocolate, 19 Oz"
6,B0000KC2BK,Knorr,3.09,Knorr Granulated Chicken Flavor Bouillon is a ...
7,B0001E1IN8,Castillo,NaN,Red chili habanero sauces. They are present to...
8,B00032E8XK,Chicken of the Sea,1.48,Chicken of the Sea Solid White Albacore Tuna i...
9,B0005XMTHE,Smucker's,2.29,"Helps build muscles with bcaa's amino acids, i..."
10,B0005XNE6E,Snapple,1.99,"At Snapple, we believe lifes a peach. Weve bee..."


Nous allons créer deux fichiers CSV d'articles. Un seul contiendra la colonne de description. Nous utiliserons chacun d'entre eux pour entraîner des modèles distincts avec la même recette, afin de pouvoir comparer les mesures hors ligne et inspecter en ligne les recommandations.

In [22]:
items_with_desc_filename = "items-with-desc.csv"
items_df.to_csv(items_with_desc_filename, index=False, float_format='%.2f')

Un autre fichier CSV d'articles avec la colonne de description supprimée.

In [23]:
items_without_desc_df = items_df[['ITEM_ID', 'BRAND', 'PRICE']]
items_without_desc_df.head()

,ITEM_ID,BRAND,PRICE
1,B00005BPJO,Milk Duds,5.0
2,B0000DIF38,Goya,NaN
3,B0000DIWNI,Saran,NaN
4,B0000DIWNZ,Saran,NaN
5,B0000GH6UG,Ibarra,NaN


In [24]:
items_without_desc_filename = "items-without-desc.csv"
items_without_desc_df.to_csv(items_without_desc_filename, index=False, float_format='%.2f')

## Créer des groupes de jeux de données et charger les jeux de données

Une fois que nous avons créé les jeux de données dont nous avons besoin, nous les chargeons dans Personalize à l'aide de tâches d'importation de jeux de données. Pour pouvoir télécharger les fichiers CSV, nous devons créer des groupes de jeux de données contenant nos deux approches de jeux de données (sans et avec descriptions), créer des schémas pour nos jeux de données et créer des jeux de données.

Nous allons commencer par créer le client SDK dont nous aurons besoin pour interagir avec Personalize.

In [25]:
personalize = boto3.client('personalize')

### Créer les groupes de jeux de données

Créons nos deux groupes de jeux de données.

In [26]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "amazon-pantry-without-desc"
)

dataset_group_without_desc_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-without-desc",
  "ResponseMetadata": {
    "RequestId": "20bd153c-ebd0-432d-9ef3-522a0d2fd8d4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:18 GMT",
      "x-amzn-requestid": "20bd153c-ebd0-432d-9ef3-522a0d2fd8d4",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [27]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "amazon-pantry-with-desc"
)

dataset_group_with_desc_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-with-desc",
  "ResponseMetadata": {
    "RequestId": "9cec53c8-28a3-40e5-bf0e-6f87a03113f7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:18 GMT",
      "x-amzn-requestid": "9cec53c8-28a3-40e5-bf0e-6f87a03113f7",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Comme la création complète des groupes de groupe de données peut prendre quelques secondes, attendons qu'ils soient tous deux ACTIFS.

In [28]:
in_progress_dataset_group_arns = [ dataset_group_without_desc_arn, dataset_group_with_desc_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for dataset_group_arn in in_progress_dataset_group_arns:
        describe_dataset_group_response = personalize.describe_dataset_group(
            datasetGroupArn = dataset_group_arn
        )
        status = describe_dataset_group_response["datasetGroup"]["status"]
        if status == "ACTIVE":
            print("Dataset group create succeeded for {}".format(dataset_group_arn))
            in_progress_dataset_group_arns.remove(dataset_group_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(dataset_group_arn))
            in_progress_dataset_group_arns.remove(dataset_group_arn)

    if len(in_progress_dataset_group_arns) <= 0:
        break
    else:
        print("At least one dataset group create is still in progress")
                
    time.sleep(10)

At least one dataset group create is still in progress
Dataset group create succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-without-desc
At least one dataset group create is still in progress
Dataset group create succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-with-desc


### Créer un schéma de jeu de données des interactions et les jeux de données

Comme le jeu de données des interactions sera le même pour les deux groupes de jeux de données, nous allons créer un seul schéma pour le type de jeux de données des interactions et le partager entre les deux groupes de jeux de données. Cela est possible, car les schémas sont globaux dans votre compte AWS et non pas spécifiques à un groupe de jeux de données.

In [29]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_VALUE",
            "type": "float"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        }
    ],
    "version": "1.0"
}
            
create_schema_response = personalize.create_schema(
    name = "amazon-pantry-interactions",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-interactions",
  "ResponseMetadata": {
    "RequestId": "6be5e019-0c8e-487b-a158-6f089f9e79ca",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:40 GMT",
      "x-amzn-requestid": "6be5e019-0c8e-487b-a158-6f089f9e79ca",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Ensuite, nous allons créer un jeu de données des interactions dans les deux groupes de jeux de données en spécifiant le schéma que nous venons de créer.

In [30]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-without-desc-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_without_desc_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_without_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-without-desc/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "ea1020bb-a7fa-4a64-a29e-f07e8e0f9ae9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:41 GMT",
      "x-amzn-requestid": "ea1020bb-a7fa-4a64-a29e-f07e8e0f9ae9",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [31]:
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-with-desc-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_with_desc_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_with_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-with-desc/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "614257cc-6344-408e-8e15-89d4f81ae927",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:41 GMT",
      "x-amzn-requestid": "614257cc-6344-408e-8e15-89d4f81ae927",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Mettre en place le CSV des interactions dans S3

Pour pouvoir télécharger le fichier CSV d'interactions que nous avons créé précédemment vers les jeux de données Personalize que nous venons de créer, nous devons le transférer vers un compartiment S3.

Créons un compartiment S3 et copions le fichier CSV des interactions sur le compartiment.

In [32]:
# Determine the current S3 region where this notebook is being hosted in SageMaker.
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


In [33]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "amazon-pantry-personalize-text"
print(bucket_name)
if region == "us-east-1":
    s3.create_bucket(Bucket=bucket_name)
else:
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
    )

224124347618-us-east-1-amazon-pantry-personalize-text


#### Charger les interactions CSV vers S3

In [34]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

### Créer une politique de compartiment S3 et un rôle IAM

Avant de pouvoir soumettre une tâche d'importation de jeux de données à Personalize, nous devons créer une politique de compartiment et un rôle IAM qui permettront à Personalize d'accéder à notre compartiment.

In [35]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'SBN10P9R7H7ST5RK',
  'HostId': 'DD8fYEx27yBq6/rB7o9lMvkdCLOHOewN05NSq73g30jeFBdouLj5D+fWSnIZHvDuAKdCKEo7w3k=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'DD8fYEx27yBq6/rB7o9lMvkdCLOHOewN05NSq73g30jeFBdouLj5D+fWSnIZHvDuAKdCKEo7w3k=',
   'x-amz-request-id': 'SBN10P9R7H7ST5RK',
   'date': 'Tue, 13 Jul 2021 22:10:59 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [36]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleAmazonPantry"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(20) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::224124347618:role/PersonalizeRoleAmazonPantry


### Importer les jeux de données des interactions de chaque groupe de jeux de données

Nous sommes maintenant prêts à importer le fichier CSV transféré des interactions dans notre compartiment S3 vers les jeux de données Personalize que nous avons créés dans chaque groupe de jeux de de données. Nous allons soumettre les deux tâches d'importation et attendre qu'elles se terminent.

In [37]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-without-desc-ints-import",
    datasetArn = interactions_dataset_without_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_without_ints_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-ints-import",
  "ResponseMetadata": {
    "RequestId": "84c4d71d-fe71-4ee7-bccf-4f8ed8b1e549",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:12:08 GMT",
      "x-amzn-requestid": "84c4d71d-fe71-4ee7-bccf-4f8ed8b1e549",
      "content-length": "126",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [38]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-with-desc-ints-import",
    datasetArn = interactions_dataset_with_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_with_ints_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-ints-import",
  "ResponseMetadata": {
    "RequestId": "eae39716-264b-48e8-bfb4-93f569c7a904",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:12:09 GMT",
      "x-amzn-requestid": "eae39716-264b-48e8-bfb4-93f569c7a904",
      "content-length": "123",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Attendre la fin des tâches d'importation des jeux de données d'Interactions

La cellule suivante attend que les deux tâches d'importation soient terminées.

In [39]:
%%time

in_progress_import_arns = [ dataset_import_job_without_ints_arn, dataset_import_job_with_ints_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for import_arn in in_progress_import_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = import_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        if status == "ACTIVE":
            print("Dataset import succeeded for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)

    if len(in_progress_import_arns) <= 0:
        break
    else:
        print("At least one dataset import job is still in progress")
                
    time.sleep(60)

At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-ints-import
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-ints-import
CPU times: user 42.3 ms, sys: 6.1 ms, total: 48.4 ms
Wall time: 5min


### Créer un schéma de jeu de données d'articles et les jeux de données

Ensuite, nous allons répéter le processus pour les jeux de données des articles. Cette fois, cependant, nous devrons créer deux schémas, car l'un des jeux de données des articles comprend la colonne de description et l'autre non. Nous allons commencer par le schéma qui n'inclut pas la description.

In [40]:
item_without_desc_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "BRAND",
            "type": [ "null", "string" ],
            "categorical": True
        },{
            "name": "PRICE",
            "type": [ "null", "float" ],
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "amazon-pantry-item-without-desc-schema",
    schema = json.dumps(item_without_desc_schema)
)

item_without_desc_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-item-without-desc-schema",
  "ResponseMetadata": {
    "RequestId": "11a18b05-189b-4485-ba8b-58083e784321",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:17:16 GMT",
      "x-amzn-requestid": "11a18b05-189b-4485-ba8b-58083e784321",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Ensuite, nous allons créer un schéma qui inclut la description. Veillez à noter l'attribut `"textual": True` dans le champ `DESCRIPTION`. C'est ainsi que vous distinguez les champs de texte non structuré des champs catégoriques et des champs de chaînes. Sans cet attribut, Personalize n'appliquera pas les techniques de traitement du langage naturel pour extraire des caractéristiques de ce texte.

In [41]:
item_with_desc_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "BRAND",
            "type": [ "null", "string" ],
            "categorical": True
        },{
            "name": "PRICE",
            "type": [ "null", "float" ],
        },{
            "name": "DESCRIPTION",
            "type": [ "null", "string" ],
            "textual": True
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "amazon-pantry-item-with-desc-schema",
    schema = json.dumps(item_with_desc_schema)
)

item_with_desc_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-item-with-desc-schema",
  "ResponseMetadata": {
    "RequestId": "05c60e80-1d7c-45f1-a881-d08af62a8432",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:17:16 GMT",
      "x-amzn-requestid": "05c60e80-1d7c-45f1-a881-d08af62a8432",
      "content-length": "101",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Ensuite, nous allons créer des jeux de données Personalize dans chaque groupe de jeux de données, en veillant à spécifier l'ARN de schéma approprié de chaque jeu de données.

In [42]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-without-desc-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_without_desc_arn,
    schemaArn = item_without_desc_schema_arn
)

items_dataset_without_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-without-desc/ITEMS",
  "ResponseMetadata": {
    "RequestId": "f9b87f6c-22c8-42a7-8a3a-8c203300e3eb",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:13 GMT",
      "x-amzn-requestid": "f9b87f6c-22c8-42a7-8a3a-8c203300e3eb",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [43]:
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-with-desc-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_with_desc_arn,
    schemaArn = item_with_desc_schema_arn
)

items_dataset_with_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-with-desc/ITEMS",
  "ResponseMetadata": {
    "RequestId": "cbba4d2a-17c9-4a9b-a6a3-a2df934e2de1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:16 GMT",
      "x-amzn-requestid": "cbba4d2a-17c9-4a9b-a6a3-a2df934e2de1",
      "content-length": "97",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Mettre en place le CSV des articles dans S3

Ensuite, nous allons copier nos deux fichiers CSV des articles vers le compartiment S3 créé ci-dessus.

In [44]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_without_desc_filename).upload_file(items_without_desc_filename)

In [45]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_with_desc_filename).upload_file(items_with_desc_filename)

### Importer des jeux de données d'articles pour chaque groupe de jeux de données

Puisque la politique du compartiment S3 et le rôle IAM sont déjà configurés, nous pouvons simplement soumettre deux tâches d'importation de jeux de données pour importer les CSV des articles.

In [46]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-without-desc-items-import",
    datasetArn = items_dataset_without_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_without_desc_filename)
    },
    roleArn = role_arn
)

dataset_import_job_without_items_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-items-import",
  "ResponseMetadata": {
    "RequestId": "c6ea207b-b8eb-4565-8ce2-6eb44e0fa18f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:37 GMT",
      "x-amzn-requestid": "c6ea207b-b8eb-4565-8ce2-6eb44e0fa18f",
      "content-length": "127",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [47]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-with-desc-items-import",
    datasetArn = items_dataset_with_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_with_desc_filename)
    },
    roleArn = role_arn
)

dataset_import_job_with_items_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-items-import",
  "ResponseMetadata": {
    "RequestId": "cf95206f-1527-4247-a618-6c8c832fa05f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:38 GMT",
      "x-amzn-requestid": "cf95206f-1527-4247-a618-6c8c832fa05f",
      "content-length": "124",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Attendre la fin des travaux d'importation des articles

La logique suivante attend que les deux jeux de données de articles soient entièrement importés vers chaque groupe de jeux de données.

In [48]:
%%time

in_progress_import_arns = [ dataset_import_job_without_items_arn, dataset_import_job_with_items_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for import_arn in in_progress_import_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = import_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        if status == "ACTIVE":
            print("Dataset import succeeded for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)

    if len(in_progress_import_arns) <= 0:
        break
    else:
        print("At least one dataset import job is still in progress")
                
    time.sleep(60)

At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-items-import
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-items-import
CPU times: user 57.6 ms, sys: 5.06 ms, total: 62.7 ms
Wall time: 7min


## Créer des solutions et des versions de solution

Une fois les jeux de données d'interactions et d'articles importés dans chaque groupe de données, nous allons ensuite créer des solutions et des versions de solutions en utilisant la recette de personnalisation de l'utilisateur pour les données de chaque groupe de jeu de données.

Tout d'abord, répertorions les recettes Personalize disponibles.

In [49]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   'stat

Nous utiliserons la recette user-personalization pour ce bloc-notes, car c'est l'une des recettes qui utilise les métadonnées des articles. Cette recette prend en charge le cas d'utilisation canonique de personnalisation où, pour un utilisateur donné, vous voulez que Personalize recommande des articles qui pourraient l'intéresser. 

In [50]:
user_personalization_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

Tout d'abord, nous allons créer une solution et une version de solution dans le groupe de jeux de données qui n'inclut pas les descriptions des articles.

In [51]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-without-desc-userpersonalization",
    datasetGroupArn = dataset_group_without_desc_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_without_desc_solution_arn = user_personalization_create_solution_response['solutionArn']

In [52]:
print(user_personalization_without_desc_solution_arn)

arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization


In [53]:
user_personalization_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_without_desc_solution_arn
)

In [54]:
user_personalization_without_solution_version_arn = user_personalization_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization/0b76212f",
  "ResponseMetadata": {
    "RequestId": "018b3fcb-10d5-4290-a17a-3970723abacd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:26:14 GMT",
      "x-amzn-requestid": "018b3fcb-10d5-4290-a17a-3970723abacd",
      "content-length": "132",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Ensuite, nous allons créer une solution et une version de solution dans le groupe de jeux de données qui inclut les descriptions des articles.

In [55]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-with-desc-userpersonalization",
    datasetGroupArn = dataset_group_with_desc_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_with_desc_solution_arn = user_personalization_create_solution_response['solutionArn']

In [56]:
print(user_personalization_with_desc_solution_arn)

arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization


In [57]:
user_personalization_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_with_desc_solution_arn
)

In [58]:
user_personalization_with_solution_version_arn = user_personalization_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization/f178990f",
  "ResponseMetadata": {
    "RequestId": "f630b862-6fa9-4eb7-a0d2-71d0b9637e80",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:26:29 GMT",
      "x-amzn-requestid": "f630b862-6fa9-4eb7-a0d2-71d0b9637e80",
      "content-length": "129",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Attendre que les versions de la solution deviennent actives

Enfin, nous attendrons que la création des versions de la solution soit terminée. À cette étape, Personalize entraîne des modèles de machine learning sur la base des jeux de données et de la recette sélectionnée. Personalize divisera également les jeux de données des interactions pour créer des partitions d'entraînement et d'évaluation, afin de d'évaluer la qualité des recommandations par rapport au modèle entraîné en utilisant les données exclues.

Vous noterez que l'entraînement de la version de la solution dans le groupe de jeux données qui inclut les données des descriptions prendra plus de temps que celle sans les descriptions.

In [59]:
%%time

in_progress_solution_versions = [
    user_personalization_without_solution_version_arn,
    user_personalization_with_solution_version_arn
]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solutio

En règle générale, l'ajout de métadonnées non structurées basées sur du texte augmente le temps d'entraînement. Dans notre cas, vous voyez ci-dessus que la version de la solution qui a été entraînée sur le jeu de données avec les descriptions des produits a pris environ 15 minutes de plus que la version de la solution entraînée sur le jeu de données sans les descriptions des produits. Cette différence variera en fonction de la composition et des valeurs de texte de vos jeux de données.

Examinons les heures d'entraînement de chaque version de la solution et comparons-les.

In [60]:
response = personalize.describe_solution_version(solutionVersionArn = user_personalization_without_solution_version_arn)
training_hours_without_desc = response['solutionVersion']['trainingHours']

response = personalize.describe_solution_version(solutionVersionArn = user_personalization_with_solution_version_arn)
training_hours_with_desc = response['solutionVersion']['trainingHours']
training_diff = (training_hours_with_desc - training_hours_without_desc) / training_hours_without_desc

print(f"Training hours without description: {training_hours_without_desc}")
print(f"Training hours with description: {training_hours_with_desc}")

print("Difference of {:.2%}".format(training_diff))

Training hours without description: 4.199
Training hours with description: 5.346
Difference of 27.32%


Le nombre d'heures d'entraînement nécessaires pour le calcul des coûts étaient environ 27 % supérieur pour les entraînements avec la colonne des descriptions. 

L'heure et les durées d'entraînement varieront en fonction de la taille de vos jeux de données, mais ces informations peuvent vous aider à évaluer le compromis à faire lorsque vous envisagez d'ajouter du texte non structuré à vos jeux de données.

### Inspecter les métriques hors ligne

Maintenant que la création des versions de la solution est terminée, inspectons et comparons les métriques hors ligne de chacune de ses versions, afin de déterminer l'impact de l'inclusion de texte non structuré sur ces métriques.

In [61]:
metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_without_solution_version_arn
)

print(json.dumps(metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization/0b76212f",
  "metrics": {
    "coverage": 0.0914,
    "mean_reciprocal_rank_at_25": 0.0268,
    "normalized_discounted_cumulative_gain_at_10": 0.0376,
    "normalized_discounted_cumulative_gain_at_25": 0.0464,
    "normalized_discounted_cumulative_gain_at_5": 0.0309,
    "precision_at_10": 0.0058,
    "precision_at_25": 0.0037,
    "precision_at_5": 0.0076
  },
  "ResponseMetadata": {
    "RequestId": "8c61339a-f929-47e0-81f0-a9660ebd589f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 23:16:07 GMT",
      "x-amzn-requestid": "8c61339a-f929-47e0-81f0-a9660ebd589f",
      "content-length": "430",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Enregistrons-les dans un dictionnaire, afin de pouvoir comparer plus facilement les métriques des deux versions de la solution.

In [62]:
metrics = {
    'Coverage': [ metrics_response['metrics']['coverage'] ],
    'MRR-25': [ metrics_response['metrics']['mean_reciprocal_rank_at_25'] ],
    'NDCG-5': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_5'] ],
    'NDCG-10': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_10'] ],
    'NDCG-25': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_25'] ],    
    'Precision-5': [ metrics_response['metrics']['precision_at_5'] ],
    'Precision-10': [ metrics_response['metrics']['precision_at_10'] ],
    'Precision-25': [ metrics_response['metrics']['precision_at_25'] ],    
}

Ensuite, récupérez les métriques hors ligne de la version de la solution qui contenait la colonne de description et enregistrez-les également.

In [63]:
metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_with_solution_version_arn
)

print(json.dumps(metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization/f178990f",
  "metrics": {
    "coverage": 0.1323,
    "mean_reciprocal_rank_at_25": 0.0367,
    "normalized_discounted_cumulative_gain_at_10": 0.049,
    "normalized_discounted_cumulative_gain_at_25": 0.0591,
    "normalized_discounted_cumulative_gain_at_5": 0.0425,
    "precision_at_10": 0.0071,
    "precision_at_25": 0.0045,
    "precision_at_5": 0.0104
  },
  "ResponseMetadata": {
    "RequestId": "b54df693-4378-4194-96c7-cec3a9d934cf",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 23:16:14 GMT",
      "x-amzn-requestid": "b54df693-4378-4194-96c7-cec3a9d934cf",
      "content-length": "426",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [64]:
metrics['Coverage'].append(metrics_response['metrics']['coverage'])
metrics['MRR-25'].append(metrics_response['metrics']['mean_reciprocal_rank_at_25'])
metrics['NDCG-5'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_5'])
metrics['NDCG-10'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_10'])
metrics['NDCG-25'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_25'])
metrics['Precision-5'].append(metrics_response['metrics']['precision_at_5'])
metrics['Precision-10'].append(metrics_response['metrics']['precision_at_10'])
metrics['Precision-25'].append(metrics_response['metrics']['precision_at_25'])

Calculez le pourcentage de changement dans chaque métrique avec ajout de texte et sans ajout de texte et affichez les résultats.

In [65]:
for key in metrics:
    metrics[key].append("{:.2%}".format((metrics[key][1] - metrics[key][0])/metrics[key][0]))

metrics_df = pd.DataFrame.from_dict(metrics,orient='index',columns=['Without Text', 'With Text', '% Change'])
metrics_df

,Without Text,With Text,% Change
Coverage,0.0914,0.1323,44.75%
MRR-25,0.0268,0.0367,36.94%
NDCG-5,0.0309,0.0425,37.54%
NDCG-10,0.0376,0.0490,30.32%
NDCG-25,0.0464,0.0591,27.37%
Precision-5,0.0076,0.0104,36.84%
Precision-10,0.0058,0.0071,22.41%
Precision-25,0.0037,0.0045,21.62%


Ces métriques montrent clairement que les recommandations de la version de la solution qui inclut les descriptions d'articles sont nettement meilleures dans tous les cas. Dans le cas où il existe moins d'interactions, les jeux de données, où les utilisateurs et les articles ont moins d'interactions, bénéficieront davantage de l'ajout de texte que les jeux de données qui ont déjà un nombre élevé d'interactions par articles et/ou utilisateur.

## Nettoyer

Les ressources Personalize créées par ce bloc-notes peuvent être supprimées à partir de la page du service Personalize dans la console AWS. 

Le script suivant peut également être exécuté localement pour supprimer toutes les ressources de chaque groupe de jeux de données.

https://gist.github.com/james-jory/62ddddf2f9180b77dd2a42e645b9d3b0

En outre, le rôle IAM et le compartiment S3 peuvent être supprimés respectivement à partir des pages des service IAM et S3.